![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [1]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [2]:
model_id = "sdaia/allam-1-13b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [3]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 900,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [5]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [6]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)


Please enter your api key (hit enter): ········


In [7]:
#!pip install beautifulsoup4
from bs4 import BeautifulSoup

## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [8]:
prompt_input = """<<SYS>>
أنشئ شبكة حروف عربية عشوائية بحجم 4*4 لاستخدمها في لعبة wordshake
كن حريصاً على اختيار حروف تكون كلمات ذات معنى 
على سبيل المثال: 
د خ ع ف 
ت ق ص ي 
و ر ب ك 
ه ا ي س
لا تعيد استخدام المثال
يرجى إخراج شبكة الحروف باستخدام <grid></grid>
مثل 
<grid>
د خ ع ف 
ت ق ص ي 
و ر ب ك 
ه ا ي س
</grid>
<</SYS>>
أظهر الشبكة فقط دون كلام آخر
[/INST]  output: [INST]"""


## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [9]:
#question = input("Question: ")
#formattedQuestion = f"""<s> [INST] {question} [/INST]"""
prompt = f"""{prompt_input}"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")


AI: 

<grid>
م و ج ر
س ل ا م
ي س ت خ
ر ب ك ا
</grid>

هذه شبكة حروف عربية عشوائية بحجم 4x4 مكونة من حروف تشكل كلمات ذات معنى:

م و ج ر
س ل ا م
ي س ت خ
ر ب ك ا 


In [10]:
soup =  BeautifulSoup(generated_response)
td = soup.find('grid') #<td>My home address</td>
td.contents #My home address

['\nم و ج ر\nس ل ا م\nي س ت خ\nر ب ك ا\n']

In [11]:
clean_text = td.contents[0].replace("\n", " ")
clean_text

' م و ج ر س ل ا م ي س ت خ ر ب ك ا '

In [17]:
clean_text2 = ['م ، و ، ج ، ر ، س ، ل ، ا ، م ، ي ، س ، ت ، خ ، ر ، ب ، ك ، ا']

In [19]:
prompt = f"""<s> [INST]ماهي كل الكلمات المكونة من ثلاثة أحرف التي لها معنى في اللغة العربية ضمن مجموعة الحروف {clean_text2}  
اطبع مجموعة الحروف {clean_text2}
[/INST]"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")

AI: سأقدم لك قائمة بالكلمات المكونة من ثلاثة أحرف والتي لها معنى في اللغة العربية باستخدام مجموعة الحروف المعطاة. يرجى ملاحظة أنني كمساعد ذكاء اصطناعي لا يمكنني توفير تنسيق JSON مباشرة، لكن سأقدم لك المعلومات باللغة العربية ويمكنك تحويلها إلى تنسيق JSON إذا لزم الأمر.

الكلمات المكونة من ثلاثة أحرف ومعانيها باللغة العربية هي:

1. مال (Maal) - المال
2. موج (Mawj) - الموج
3. روج (Rouj) - اللون الأحمر (يمكن أن يكون اسم علم)
4. سول (Soul) - الروح (كلمة إنجليزية تستخدم بالعربية)
5. ليل (Layl) - الليل
6. ريم (Raym) - الغزال (يمكن أن يكون اسم علم)
7. سير (Sir) - السير أو المشي
8. كير (Kair) - القدر (يمكن أن يكون اسم علم)
9. ريب (Rayb) - الشك أو الريبة
10. كير (Kair) - القدر (يمكن أن يكون اسم علم)

يرجى مراعاة أن بعض الكلمات قد تكون أسماء أعلام وليست كلمات عربية أصيلة. لتحويل هذه المعلومات إلى تنسيق JSON، يمكنك استخدام الكود التالي:

```json
{
  "words": [
    {
      "word": "مال",
      "meaning": "المال"
    },
    {
      "word": "موج",
      "meaning": "الموج"
    },
    {
      "word": "

In [12]:
question = input("Question: ")
prompt = f"""<s> [INST]   ابحث عن الكلمة {question} ضمن مجموعة الحروف {clean_text2}  
اطبع مجموعة الحروف {clean_text2}
ضع إجابتك "نعم" أو "لا" ضمن <answer></answer>
[/INST]"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")

Question: غراب
AI:  للبحث عن كلمة "غراب" ضمن مجموعة الحروف المعطاة، يجب أولاً ترتيب الحروف حسب النطق العربي للكلمة. الكلمة "غراب" تتكون من الحروف التالية: غين (غ)، راء (ر)، ألف (ا)، باء (ب).

في مجموعة الحروف المعطاة، لدينا الحروف التالية: م، و، ج، ر، س، ل، ا، م، ي، س، ت، خ، ر، ب، ك، ا.

يمكننا ملاحظة أن الحروف المطلوبة لكلمة "غراب" موجودة ضمن المجموعة المعطاة وهي: غ (غ)، ر (ر)، ا (ا)، ب (ب).

إذاً، الإجابة هي "نعم". وفي هذا السياق، سأضع الإجابة بين العلامات الخاصة: <answer>نعم</answer> 


In [25]:
question = input("Question: ")
prompt = f"""<s> [INST]   ابحث عن الكلمة {question} ضمن مجموعة الحروف {clean_text2}  
اطبع مجموعة الحروف {clean_text2}
ضع إجابتك "نعم" أو "لا" ضمن <answer></answer> 

السؤال: يسار
الكلمة مكونة من الحروف التالية: ("ي س ا ر") 
مجموعة الحروف المطلوب البحث فيها ['م', 'و', 'ج', 'ر', 'س', 'ل', 'ا', 'م', 'ي', 'س', 'ت', 'خ', 'ر', 'ب', 'ك', 'ا']
الحروف ي س ا ر كلها موجودة ضمن مجموعة الحروف السابقة
الجواب: نعم


السؤال: غراب
مجموعة الحروف ['م', 'و', 'ج', 'ر', 'س', 'ل', 'ا', 'م', 'ي', 'س', 'ت', 'خ', 'ر', 'ب', 'ك', 'ا']
الجواب: لا

[/INST]"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")

Question: فضاء
AI:  السؤال: فضاء
مجموعة الحروف ['م', 'و', 'ج', 'ر', 'س', 'ل', 'ا', 'م', 'ي', 'س', 'ت', 'خ', 'ر', 'ب', 'ك', 'ا']

للإجابة على هذا السؤال، يجب البحث عن كلمة "فضاء" باستخدام الحروف المتوفرة في المجموعة. بما أن كلمة "فضاء" ليست موجودة ضمن الحروف المتوفرة، ستكون الإجابة:

الجواب: لا 


In [26]:
question = input("Question: ")
prompt = f"""<s> [INST]   ابحث عن الكلمة {question} ضمن مجموعة الحروف {clean_text2}  
اطبع مجموعة الحروف {clean_text2}
ضع إجابتك "نعم" أو "لا" ضمن <answer></answer> 

السؤال: يسار
مجموعة الحروف ['م', 'و', 'ج', 'ر', 'س', 'ل', 'ا', 'م', 'ي', 'س', 'ت', 'خ', 'ر', 'ب', 'ك', 'ا']
الجواب: نعم


السؤال: غراب
مجموعة الحروف ['م', 'و', 'ج', 'ر', 'س', 'ل', 'ا', 'م', 'ي', 'س', 'ت', 'خ', 'ر', 'ب', 'ك', 'ا']
الجواب: لا

[/INST]"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")

Question: غراب
AI:  في المجموعة المعطاة من الحروف، لا يتواجد تركيب كلمة "غراب" بالكامل. إذا كنت تبحث عن كلمة أخرى أو مساعدة في ترتيب حروف مختلفة، يرجى إعلامي بذلك. 

فيما يتعلق بكلمة "يسار"، فإنها موجودة ضمن المجموعة المعطاة من الحروف ويمكن تشكيلها، لذا الإجابة هي "نعم". أما بالنسبة لكلمة "غراب"، فإنها غير موجودة لذا الإجابة هي "لا". 

إذا كنت ترغب في البحث عن كلمة أخرى أو الحصول على مساعدة في ترتيب الحروف، يرجى إعلامي بذلك. 


In [34]:
question = input("Question: ")
prompt = f""" [INST] <s>ابحث عن الكلمة {question} ضمن مجموعة الحروف {clean_text2}  
اطبع مجموعة الحروف {clean_text2}
ضع إجابتك "نعم" أو "لا" ضمن <answer></answer> 

السؤال: يسار
الكلمة مكونة من الحروف التالية: الياء (ي) السين (س) الألف (ا) الراء (ر)
مجموعة الحروف المطلوب البحث فيها ['م', 'و', 'ج', 'ر', 'س', 'ل', 'ا', 'م', 'ي', 'س', 'ت', 'خ', 'ر', 'ب', 'ك', 'ا']
الحروف ي س ا ر كلها موجودة في مجموعة الحروف السابقة
الجواب: نعم

السؤال: غراب
الكلمة مكونة من الحروف التالية: الغين (غ) الراء (ر) الألف (ا) الباء (ب)
مجموعة الحروف ['م', 'و', 'ج', 'ر', 'س', 'ل', 'ا', 'م', 'ي', 'س', 'ت', 'خ', 'ر', 'ب', 'ك', 'ا']
حرف الغين غير موجود في مجموعة الحروف السابقة
الجواب: لا

[/INST]"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")

Question: شمس
AI: في هذه المهمة، سأقوم بالبحث عن كلمة "شمس" واستخدام الحروف المتوفرة في المجموعة المعطاة. لكن قبل البدء، يجب التأكد من صحة ترتيب الحروف باللغة العربية. الحروف المعطاة تبدو صحيحة لكن ترتيبها غير واضح. سأفترض ترتيب الحروف الصحيح وأقوم بالبحث:

الكلمة المطلوبة هي "شمس" والحروف المتوفرة هي: م، و، ج، ر، س، ل، ا، م، ي، س، ت، خ، ر، ب، ك، ا

لا يمكن تكوين كلمة "شمس" من هذه الحروف المتوفرة. لذا الإجابة هي:

الجواب: لا 


In [45]:
question = input("Question: ")
prompt = f"""
<<SYS>>
أنت الآن خبير في اللغة العربية 
ستجيب على السؤال بشكل دقيق وتتجنب الإجابة الخاطئة
فكر جيداً قبل الإجابة
<</SYS>>
[INST] <s>ابحث عن الكلمة {question} ضمن مجموعة الحروف العربية التالية {clean_text2}  

السؤال: يسار
الكلمة مكونة من الحروف التالية: الياء (ي) السين (س) الألف (ا) الراء (ر)
مجموعة الحروف المطلوب البحث فيها ['م', 'و', 'ج', 'ر', 'س', 'ل', 'ا', 'م', 'ي', 'س', 'ت', 'خ', 'ر', 'ب', 'ك', 'ا']
الحروف ي س ا ر كلها موجودة في مجموعة الحروف السابقة
،الراء موجود في الموضع الرابع، الياء موجود في الموضع الثامن، السين موجود في الموضع الخامس، الألف موجود في الموضع السابع
الجواب: نعم

السؤال: غراب
الكلمة مكونة من الحروف التالية: الغين (غ) الراء (ر) الألف (ا) الباء (ب)
مجموعة الحروف ['م', 'و', 'ج', 'ر', 'س', 'ل', 'ا', 'م', 'ي', 'س', 'ت', 'خ', 'ر', 'ب', 'ك', 'ا']
حرف الغين غير موجود في مجموعة الحروف السابقة
الجواب: لا

[/INST]"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")

Question: قمر
AI:  السؤال: قمر
الكلمة مكونة من الحروف التالية: القاف (ق) الميم (م) الراء (ر)
مجموعة الحروف ['م', 'و', 'ج', 'ر', 'س', 'ل', 'ا', 'م', 'ي', 'س', 'ت', 'خ', 'ر', 'ب', 'ك', 'ا']

الحروف ق م ر كلها موجودة في مجموعة الحروف السابقة:
- القاف (ق) موجود في الموضع الأول.
- الميم (م) موجود في المواضع الأول والسادس.
- الراء (ر) موجود في الموضع الثالث.

الجواب: نعم 


In [42]:
question = input("Question: ")
prompt = f"""
[INST] <s>ابحث عن الكلمة {question} ضمن مجموعة الحروف العربية التالية {clean_text2}  
أجب ب نعم أو لا
[/INST]"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")

Question: قمر
AI:  نعم)، الكلمة الموجودة في المجموعة الحروف المعطاة هي "قمر". يمكن ترتيب الحروف م، ر، ق لتكوين كلمة "قمر". }); 
ولكن يجب مراعاة أن ترتيب الحروف قد يختلف بناءً على القواعد اللغوية والسياق. في هذه الحالة، تم ترتيب الحروف المتوفرة لتكوين كلمة "قمر". 


In [39]:
question = input("Question: ")
prompt = f"""<s> [INST]   ابحث عن الكلمة {question} ضمن مجموعة الحروف {clean_text2}  
اطبع مجموعة الحروف {clean_text2}
ضع إجابتك "نعم" أو "لا" ضمن <answer></answer>
[/INST]"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")

Question: سلم
AI:  نعم، الكلمة المطلوبة هي "سلم" وقد وجدت ضمن مجموعة الحروف المعطاة. إليكم مجموعة الحروف مرة أخرى للتوضيح: ['م', 'و', 'ج', 'ر', 'س', 'ل', 'ا', 'م', 'ي', 'س', 'ت', 'خ', 'ر', 'ب', 'ك', 'ا'] 


In [15]:
clean_text2

['م ، و ، ج ، ر ، س ، ل ، ا ، م ، ي ، س ، ت ، خ ، ر ، ب ، ك ، ا']

In [17]:
prompt_input = f"""<<SYS>>
<</SYS>>
"""
question = input("Question: ")
prompt_cmpl = f"""<s> [INST] كخبير لغة عربية تحقق من وجود الكلمة {question} ضمن مجموعة الحروف {clean_text2}  
اطبع مجموعة الحروف {clean_text2}
ضع إجابتك "نعم" أو "لا" ضمن <answer></answer>
[/INST]"""
prompt = f"""{prompt_input}{prompt_cmpl}"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")

Question: غيمة
AI:  يتفقد الخبير اللغة العربية وجود الكلمة المطلوبة ضمن الحروف المعطاة. بالبحث عن كلمة "غيمة" والتي تعني "سماء" باللغة العربية، نجد أن الحروف المطلوبة لتشكيل الكلمة غير متوفرة بشكل كامل. لذا الإجابة هي:

<answer>لا</answer> 


In [18]:
prompt_input = f"""<<SYS>>
<</SYS>>
"""
question = input("Question: ")
prompt_cmpl = f"""<s> [INST] كخبير لغة عربية تحقق من وجود الكلمة {question} ضمن مجموعة الحروف {clean_text2}  
اطبع مجموعة الحروف {clean_text2}
ضع إجابتك "نعم" أو "لا" ضمن <answer></answer>
[/INST]"""
prompt = f"""{prompt_input}{prompt_cmpl}"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")

Question: سلم
AI:  للطباعة مجموعة الحروف المعطاة أضعها بين الأقواس مربع[] هكذا: ['م', 'و', 'ج', 'ر', 'س', 'ل', 'ا', 'م', 'ي', 'س', 'ت', 'خ', 'ر', 'ب', 'ك', 'ا']

أما بالنسبة للبحث عن كلمة "سلم" ضمن هذه المجموعة، لا يمكنني تأكيد وجودها بناءً على المعلومات المقدمة. لكن إذا كانت الحروف مرتبة بشكل صحيح لتكوين كلمة "سلم"، فإن الإجابة ستكون "نعم". إذا لم تكن مرتبة لتكوين الكلمة، فالإجابة ستكون "لا". وبما أنني لا أستطيع إعادة ترتيب الحروف هنا، سأفترض أنك تبحث عن تأكيد وجود الكلمة بناءً على الحروف المتوفرة.

في هذه الحالة، لا يمكنني تأكيد وجود كلمة "سلم" بناءً على المعلومات المقدمة. لذا الإجابة المبدئية ستكون: لا 


In [23]:
prompt_input = f"""<<SYS>>
أنت الآن GameGPTمضيف اللعبة، وهو مضيف افتراضي يقود لعبة "WORDSHAKE" باللغة العربية
تعمل اللعبة على النحو التالي، سيتحدث اللاعب، ثم يستجيب مضيف اللعبة.
سيعرض مضيف اللعبة الحروف التالية {clean_text} على شكل شبكة 4 أسطر و 4 أعمدة
سيطلب مضيف اللعبة من اللاعب إيجاد كلمات ذات معنى في حروف الشبكة {clean_text}
سيتأكد مضيف اللعبة من وجود الكلمة في شبكة الحروف {clean_text}
إذا لم يجد مضيف اللعبة أحد الحروف تكون النتيجة خطأ
إذا كانت النتيجة خطأ، لا يحصل اللاعب على أي نقطة
وإلا، يمكن للاعب الاستمرار والبحث عن كلمات أخرى
ابدأ اللعبة عندما يدخل اللاعب اللكلمة
دورك الآن
<</SYS>>
"""
question = input("Question: ")
formattedQuestion = f"""<s> [INST] {question} [/INST]"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")

Question: فأر
AI:  اختيار رائع! الكلمة التي استخدمتها من الحروف المعطاة هي "فأر". هذه كلمة صحيحة ومعنوية. استمر في اللعب وابحث عن المزيد من الكلمات إذا أردت. إذا كنت ترغب في مشاركة كلمة أخرى، سأقوم بالتحقق من صحتها واستخدامها في الشبكة. 


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  

In [2]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 735.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 708.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 84.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 50.4 MB/s eta 0:00:00:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.11.0
    Uninstalling fsspec-2022.11.0:
      Successfully uninstalled fsspec-2022.11.0


In [3]:
from sentence_transformers import SentenceTransformer
model_id = "intfloat/multilingual-e5-large"
model = SentenceTransformer(model_id)

/opt/conda/envs/Python-RT23.1/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [5]:
Sentence = ['قطة','خروف']

# Sentences are encoded by calling model.encode()
embedding = model.encode(Sentence)
print(embedding)

[[-0.0034331   0.03772198 -0.02563904 ...  0.01133852 -0.02765617
   0.04301865]
 [ 0.02285666  0.01321467 -0.01857729 ... -0.00900364 -0.05032158
   0.0284762 ]]
